# Anastomotic Leakage Prediction
### Phase 2 - ML Model

https://www.kaggle.com/prashant111/naive-bayes-classifier-in-python

In [177]:
import pandas as pd
import numpy as np
import re
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  
from datetime import datetime
from datetime import date 
from scipy import stats
import statsmodels.stats as ss 

import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

### Import data

In [178]:
# RAW DATA
before_surgery_data = pd.read_excel(r'../generated_data/raw-data-before-surgery.xlsx')
# after_surgery_data = pd.read_excel(r'../generated_data/raw-data-after-surgery.xlsx')
# delta = pd.read_excel(r'../generated_data/raw-data-delta-surgery.xlsx')

# ALL FEATURES STATS (CV AND PVALUE)
# stats_before_surgery_data = pd.read_excel(r'../generated_data/stats-before-surgery.xlsx')
# stats_after_surgery_data = pd.read_excel(r'../generated_data/stats-after-surgery.xlsx')
# stats_surgery_data_delta = pd.read_excel(r'../generated_data/stats-delta-surgery.xlsx')

# AFTER WRS STATS (CV AND PVALUE
before_surgery_wrs = pd.read_excel(r'../generated_data/wrs-before-surgery.xlsx')
# after_surgery_final_features_corr = pd.read_excel(r'../generated_data/stats-wrs-after-surgery.xlsx')
# delta_surgery_final_features_corr = pd.read_excel(r'../generated_data/stats-wrs-delta-surgery.xlsx')

# FDR STATS (CV AND PVALUE
# stats_before_surgery_data_fdr = pd.read_excel((r'../generated_data/stats-fdr-before-surgery.xlsx', index = False)
# stats_after_surgery_data_fdr = pd.read_excel((r'../generated_data/stats-fdr-after-surgery.xlsx', index = False)
# stats_surgery_data_delta_fdr = pd.read_excel((r'../generated_data/stats-fdr-delta-surgery.xlsx', index = False)


### Model Functions

In [196]:
#  train_test_split(before_surgery_data.iloc[:,1:], before_surgery_data.iloc[:,0:1], test_size=0.33, random_state=42)
def split_data(X,y,fillna):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
    if fillna == 'mean':
        X_test.fillna(X_test.mean(),inplace=True)
        X_train.fillna(X_train.mean(),inplace=True)
    return X_train, X_test, y_train, y_test

In [208]:
# Train a Gaussian Naive Bayes classifier on the training set
def nb_model(X_train,y_train):
    # instantiate the model
    gnb = GaussianNB()
    # fit the model
    gnb.fit(X_train, y_train)
    return gnb

def predict_get_accuracy(X_train, X_test, y_train, y_test):
    y_pred = gnb.predict(X_test)
    print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
    y_pred_train = gnb.predict(X_train)
    print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
#     print('Training set score: {:.4f}'.format(gnb.score(X_train, y_train)))
#     print('Test set score: {:.4f}'.format(gnb.score(X_test, y_test)))
    return #y_pred_train, y_pred_test

#### Models training

##### Before Surgery Only - Removing data with missing values, all features

In [226]:
data = before_surgery_data.dropna(how='any')
print(f'Data Shape {data.shape}')
X_train, X_test, y_train, y_test = split_data(data.drop('Anastomotic Leak', axis=1),
                                              data['Anastomotic Leak'],
                                              fillna = False)
nb_model(X_train,y_train)
predict_get_accuracy(X_train, X_test, y_train, y_test)

Data Shape (229, 54)
Model accuracy score: 0.6974
Training-set accuracy score: 0.6536


##### Before Surgery Only - Replacing missing values with mean, all features

In [228]:
data = before_surgery_data
print(f'Data Shape {data.shape}')
X_train, X_test, y_train, y_test = split_data(data.drop('Anastomotic Leak', axis=1),
                                              data['Anastomotic Leak'],
                                              fillna = 'mean')
nb_model(X_train,y_train)
predict_get_accuracy(X_train, X_test, y_train, y_test)

Data Shape (524, 54)
Model accuracy score: 0.7399
Training-set accuracy score: 0.7806


##### Before Surgery Only - Removing data with missing values, WRS features

In [220]:
before_surgery_wrs.head()

HCT   BMI  LYMPHO (%)  Estimated Globulin    Amylase  \
0        NaN   NaN         NaN                 NaN        NaN   
1  41.040000  38.8   35.600000            3.000000  53.000000   
2  35.210000   NaN   27.600000            2.600000        NaN   
3  29.750714  23.6    8.014286            2.392857  32.692308   
4        NaN   NaN         NaN                 NaN        NaN   

   Bilirubin, direct  Phosphorus    Chloride  heart rate  Urea  ...  \
0                NaN         NaN         NaN         NaN   NaN  ...   
1               0.06    3.200000  102.000000   75.500000  24.0  ...   
2                NaN    3.600000  101.000000   89.000000  28.0  ...   
3               0.18    2.671429  100.142857   94.351145  37.0  ...   
4                NaN         NaN         NaN  105.000000   NaN  ...   

         RDW       PCT  Potassium  MONO (%)        MCV  saturation  Magnesium  \
0        NaN       NaN        NaN       NaN        NaN         NaN        NaN   
1  14.680000  0.230000   4.300000  6.600000  86.440000   99.000000   2.200000   
2  15.205152  0.260000   4.300000  6.300000  71.570000   98.000000        NaN   
3  14.456429  0.241429   3.660263  5.021429  83.272857   96.309524   2.038462   
4        NaN       NaN   3.460000       NaN        NaN   95.000000        NaN   

   Nucleated RBC (%)  Nucleated RBC (abs)  Anastomotic Leak  
0                NaN                  NaN                 0  
1           0.000000             0.000000                 0  
2           0.000000             0.000000                 0  
3           0.685714             0.021429                 0  
4                NaN                  NaN                 0  

[5 rows x 34 columns]

In [241]:
data = before_surgery_wrs.dropna(how='any')
print(f'Data Shape {data.shape}')
X_train, X_test, y_train, y_test = split_data(data.drop('Anastomotic Leak', axis=1),
                                              data['Anastomotic Leak'],
                                              fillna = False)
nb_model(X_train,y_train)

Data Shape (231, 34)


GaussianNB(priors=None, var_smoothing=1e-09)

In [242]:
predict_get_accuracy(X_train, X_test, y_train, y_test)

ValueError: operands could not be broadcast together with shapes (77,33) (53,) 

##### Before Surgery Only - Replacing data with mean, WRS features

In [236]:
data = before_surgery_wrs
print(f'Data Shape {data.shape}')
data.head()
X_train, X_test, y_train, y_test = split_data(data.drop('Anastomotic Leak', axis=1),
                                              data['Anastomotic Leak'],
                                              fillna = 'mean')
# print(X_train,y_train)
nb_model(X_train,y_train)

Data Shape (524, 34)


GaussianNB(priors=None, var_smoothing=1e-09)

In [239]:
X_test
predict_get_accuracy(X_train, X_test, y_train, y_test)

ValueError: operands could not be broadcast together with shapes (173,33) (53,) 

##### Before and After Surgery Only - Removing data with missing values, ALL features

In [240]:
# before_after_events = pd.concat([before_surgery_data.add_suffix('_BS'), after_surgery_data.iloc[:,1:].add_suffix('_AS')], axis=1)
# before_after_events

# data = before_after_events
# print(f'Data Shape {before_after_events.shape}')
# data

# X_train, X_test, y_train, y_test = split_data(data.drop('Anastomotic Leak', axis=1),
#                                               data['Anastomotic Leak'],
#                                               fillna = 'mean')
# nb_model(X_train,y_train)
# predict_get_accuracy(X_train, X_test, y_train, y_test)